In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula as smf
import numpy as np
import datetime

In [3]:
nombres = ["ANO_DEF","FECHA_DEF","GLOSA_SEXO","EDAD_TIPO","EDAD_CANT","CODIGO_COMUNA_RESIDENCIA","GLOSA_COMUNA_RESIDENCIA","GLOSA_REG_RES","DIAG1","CAPITULO_DIAG1","GLOSA_CAPITULO_DIAG1","CODIGO_GRUPO_DIAG1","GLOSA_GRUPO_DIAG1","CODIGO_CATEGORIA_DIAG1","GLOSA_CATEGORIA_DIAG1","CODIGO_SUBCATEGORIA_DIAG1","GLOSA_SUBCATEGORIA_DIAG1","DIAG2","CAPITULO_DIAG2","GLOSA_CAPITULO_DIAG2","CODIGO_GRUPO_DIAG2","GLOSA_GRUPO_DIAG2","CODIGO_CATEGORIA_DIAG2","GLOSA_CATEGORIA_DIAG2","CODIGO_SUBCATEGORIA_DIAG2","GLOSA_SUBCATEGORIA_DIAG2"]
df = pd.read_csv("prueba.csv", ";" , names = nombres)
df.dropna( inplace = True)


In [4]:
df = pd.read_csv("prueba.csv", ";" , names = nombres)
df['FECHA_DEF'] = pd.to_datetime(df['FECHA_DEF']) - pd.to_timedelta(7, unit='d')
df = df.groupby(['GLOSA_GRUPO_DIAG1','CODIGO_COMUNA_RESIDENCIA', pd.Grouper(key='FECHA_DEF', freq='W-MON')])['EDAD_TIPO'].sum().reset_index().sort_values('FECHA_DEF')


#### En esta parte tomé el dataset de las muertes diarias en Chile desde el 2016 hasta el 2021 y agrupé los datos por semanas, usé la columna edad tipo para sumar, ya que esta contaba con un uno en todas los datos.
#### Luego obtuve solo las muertes por covid usando 'Asignación provisoria de nuevas afecciones de etiología incierta' que era como se le llama a las muertes por covid Confirmadas  y no confirmadas

In [5]:
M_Covid= df[df['GLOSA_GRUPO_DIAG1'] == 'Asignación provisoria de nuevas afecciones de etiología incierta']
M_Covid

,GLOSA_GRUPO_DIAG1,CODIGO_COMUNA_RESIDENCIA,FECHA_DEF,EDAD_TIPO
9927,Asignación provisoria de nuevas afecciones de ...,5401,2019-12-30,3
9902,Asignación provisoria de nuevas afecciones de ...,5303,2019-12-30,1
10028,Asignación provisoria de nuevas afecciones de ...,5501,2019-12-30,1
9887,Asignación provisoria de nuevas afecciones de ...,5302,2019-12-30,1
10096,Asignación provisoria de nuevas afecciones de ...,5502,2019-12-30,1
...,...,...,...,...
13419,Asignación provisoria de nuevas afecciones de ...,9109,2021-11-29,1
9373,Asignación provisoria de nuevas afecciones de ...,4106,2021-11-29,3
12673,Asignación provisoria de nuevas afecciones de ...,8110,2021-11-29,2
9322,Asignación provisoria de nuevas afecciones de ...,4102,2021-11-29,8


#### En esta parte obtuve las muertes solo de la región Metropolitana 

In [8]:
dfe = M_Covid.loc[M_Covid['CODIGO_COMUNA_RESIDENCIA'].isin([13101,13605])]
dfe

,GLOSA_GRUPO_DIAG1,CODIGO_COMUNA_RESIDENCIA,FECHA_DEF,EDAD_TIPO
14596,Asignación provisoria de nuevas afecciones de ...,13101,2019-12-30,13
17795,Asignación provisoria de nuevas afecciones de ...,13605,2019-12-30,5
17796,Asignación provisoria de nuevas afecciones de ...,13605,2020-01-06,4
14597,Asignación provisoria de nuevas afecciones de ...,13101,2020-01-06,13
14598,Asignación provisoria de nuevas afecciones de ...,13101,2020-01-13,3
...,...,...,...,...
14669,Asignación provisoria de nuevas afecciones de ...,13101,2021-10-25,1
14670,Asignación provisoria de nuevas afecciones de ...,13101,2021-11-01,1
17862,Asignación provisoria de nuevas afecciones de ...,13605,2021-11-01,1
17863,Asignación provisoria de nuevas afecciones de ...,13605,2021-11-29,2


In [9]:
df3 = dfe[['FECHA_DEF','CODIGO_COMUNA_RESIDENCIA','GLOSA_GRUPO_DIAG1','EDAD_TIPO']]


#### En esta parte introducí el dataset de las vacunas por día, agrupe los datos por semana y obtuve solo los datos de la región Metropolitana

In [10]:
DFVacunas = pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/input/Vacunacion/WORK_ARCHIVO_1.csv", ";" ,encoding='ISO-8859-1')
dfv = DFVacunas.loc[DFVacunas['COD_COMUNA_FINAL'].isin([13101,13605])]
dfv['FECHA_INMUNIZACION'] = pd.to_datetime(dfv['FECHA_INMUNIZACION']) - pd.to_timedelta(7, unit='d')
dfv1 = dfv.groupby(['COD_COMUNA_FINAL', pd.Grouper(key='FECHA_INMUNIZACION', freq='W-MON')])['SUM_of_SUM_of_1aDOSIS'].sum().reset_index().sort_values('FECHA_INMUNIZACION')
dfv2 = dfv1.set_index(['FECHA_INMUNIZACION'])
dfv2

<ipython-input-10-285bc5432a33>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfv['FECHA_INMUNIZACION'] = pd.to_datetime(dfv['FECHA_INMUNIZACION']) - pd.to_timedelta(7, unit='d')


,COD_COMUNA_FINAL,SUM_of_SUM_of_1aDOSIS
FECHA_INMUNIZACION,,
2020-12-21,13101,1228.0
2020-12-28,13101,7643.0
2020-12-28,13605,1283.0
2021-01-04,13101,2788.0
2021-01-04,13605,806.0
...,...,...
2021-10-04,13101,5087.0
2021-11-01,13101,13168.0
2021-11-01,13605,3177.0


### Acá deje el dataframe listo para poder ser unido con el de las muertes totales

In [11]:
dfvac= dfv2[["SUM_of_SUM_of_1aDOSIS","COD_COMUNA_FINAL"]]
dfvac

,SUM_of_SUM_of_1aDOSIS,COD_COMUNA_FINAL
FECHA_INMUNIZACION,,
2020-12-21,1228.0,13101
2020-12-28,7643.0,13101
2020-12-28,1283.0,13605
2021-01-04,2788.0,13101
2021-01-04,806.0,13605
...,...,...
2021-10-04,5087.0,13101
2021-11-01,13168.0,13101
2021-11-01,3177.0,13605


#### En esta parte obtuve los datos de muertes desde el día en que se empezó a vacunar en Chile.

In [12]:
dfina = df3.set_index(['FECHA_DEF'])
dfex = dfina.loc['2020-12-24':'2021-11-29']
dfex

,CODIGO_COMUNA_RESIDENCIA,GLOSA_GRUPO_DIAG1,EDAD_TIPO
FECHA_DEF,,,
2020-12-28,13101,Asignación provisoria de nuevas afecciones de ...,10
2021-01-11,13605,Asignación provisoria de nuevas afecciones de ...,2
2021-01-18,13605,Asignación provisoria de nuevas afecciones de ...,1
2021-01-18,13101,Asignación provisoria de nuevas afecciones de ...,8
2021-01-25,13101,Asignación provisoria de nuevas afecciones de ...,15
2021-01-25,13605,Asignación provisoria de nuevas afecciones de ...,2
2021-02-01,13101,Asignación provisoria de nuevas afecciones de ...,4
2021-02-08,13101,Asignación provisoria de nuevas afecciones de ...,3
2021-02-15,13605,Asignación provisoria de nuevas afecciones de ...,1


#### En esta parte introducí el dataset de los indicadores de calidad de vida de la región Metropolitana, y dejé solo el código de la comuna y el HDI, para ser agregado al dataframe final.

In [17]:
DFriqueza = pd.read_csv("https://raw.githubusercontent.com/leoferres/gitiele356b/main/data/hdi.csv")
DFriqueza.dropna(inplace = True)
Dfr = DFriqueza[["comuna","HDI"]]
Dfr.head(5)

,comuna,HDI
0,13101,0.861222
1,13102,0.764616
2,13103,0.718874
3,13104,0.749584
4,13105,0.742015


### En esta parte intenté mezclar los dos dataframe, pero no pude resolver el error

In [13]:
dffinal = pd.concat([dfex,dfvac], axis = 1)

ValueError: Shape of passed values is (137, 5), indices imply (88, 5)

#### Con este codigo pretendía realizar la regresión teniendo en consideración a las vacunas suministradas como variable independiente de la regresión

In [ ]:
model = smf.ols(formula='entr_f ~ SUM_of_SUM_of_1aDOSIS', data=df)

#### Con este codigo pretendía obtener los resultados de la regresión

In [ ]:
results = model.fit()